In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
import random
from load_dataset import Make_Dataset , train_test_split
from word2vec import read_word2vec
from trainer import train
from inference_function import softmax  

In [2]:
word_embedding_dir="glove.6B.300d.txt"
csv_path ="Dataset/Tweets.csv"

maxSequence = 31
train_percentage =0.8

In [3]:
csv_file = pd.read_csv(csv_path)
word_embedding = read_word2vec(word_embedding_dir)  
data , labels = Make_Dataset(csv_file,word_embedding,maxSequence)
train_data,train_labels,val_data,val_labels = train_test_split(data,labels,train_percentage)

In [6]:
wordembedding_len = list(word_embedding.values())[0].shape[0]
numClasses =3
num_of_epochs=6
batch_size=32
train(maxSequence ,wordembedding_len,numClasses,num_of_epochs , batch_size , train_data,train_labels,val_data,val_labels)

epoch: 0 , Training accuracy:: 0.6617998480796814 , Validation accuracy:: 0.693647563457489 
epoch: 1 , Training accuracy:: 0.7677595615386963 , Validation accuracy:: 0.7182376980781555 
epoch: 2 , Training accuracy:: 0.8044740557670593 , Validation accuracy:: 0.6970628499984741 
epoch: 3 , Training accuracy:: 0.8419569730758667 , Validation accuracy:: 0.6915983557701111 
epoch: 4 , Training accuracy:: 0.8712431788444519 , Validation accuracy:: 0.7226775884628296 
epoch: 5 , Training accuracy:: 0.891308069229126 , Validation accuracy:: 0.7196038365364075 


### This shows other evaluation metrics on the validaion dataset and a confusion matrix

In [9]:
with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph('Best_model/checkpoint_5.ckpt.meta')
    new_saver.restore(sess, tf.train.latest_checkpoint('Best_model/'))
    
    with tf.Session() as sess:    
        saver = tf.train.import_meta_graph('ckpt/checkpoint_5.ckpt.meta')
        saver.restore(sess,tf.train.latest_checkpoint('ckpt/'))

        graph = tf.get_default_graph()
        input_x = graph.get_tensor_by_name("input_data:0")
        keep_prob = graph.get_tensor_by_name("keep_prob1:0")
        keep_prob2 = graph.get_tensor_by_name("keep_prob2:0")

        op_to_restore = graph.get_tensor_by_name("output:0")


        predict =sess.run(op_to_restore,feed_dict={input_x: val_data ,
                                    keep_prob:1 ,
                                    keep_prob2:1 })
        
        

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from Best_model/checkpoint_5.ckpt
INFO:tensorflow:Restoring parameters from ckpt/checkpoint_5.ckpt


In [19]:
predict_classes = softmax(predict).argmax(axis=1)

In [18]:
labels_classes = val_labels.argmax(axis=1)

In [37]:
from sklearn.metrics import confusion_matrix
print("this is the confusion matrix for the three classes")
confusion_matrix(predict_classes, labels_classes)

this is the confusion matrix for the three classes


array([[ 379,   69,  273],
       [  92,  311,  120],
       [ 179,   88, 1417]])

In [33]:
from sklearn.metrics import f1_score
f1 = f1_score(labels_classes, predict_classes, average='macro')
print("f1 score: {}".format(f1))

f1 score: 0.6638782330791724


In [34]:
from sklearn.metrics import recall_score
recal = recall_score(labels_classes, predict_classes, average='macro') 
print("recal score: {}".format(recal))

recal score: 0.6768265885945444


In [36]:
from sklearn.metrics import precision_score
percision = precision_score(labels_classes, predict_classes, average='macro') 
print("percision score: {}".format(percision))

percision score: 0.6539180032797037
